<a href="https://colab.research.google.com/github/prasanth59/Work/blob/master/Final_Post_Wise_chunking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install contractions
!pip install langdetect

     |████████████████████████████████| 327kB 4.3MB/s 
     |████████████████████████████████| 266kB 6.2MB/s 
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.2-cp37-cp37m-linux_x86_64.whl size=85392 sha256=df3b9476a938824f7b4b808c36a8b345f7633a3c97a50b1c846d0fdc2b2fa64d
  Stored in directory: /root/.cache/pip/wheels/3a/03/34/77e3ece0bba8b86bfac88a79f923b36d805cad63caeba38842
Successfully built pyahocorasick
     |████████████████████████████████| 983kB 6.1MB/s 
  Created wheel for langdetect: filename=langdetect-1.0.8-cp37-none-any.whl size=993193 sha256=fcb71bea7a1a7a6ec0b6406ea4d6d746d2de5586b1d4f3a7bce9e872dbd66327
  Stored in directory: /root/.cache/pip/wheels/8d/b3/aa/6d99de9f3841d7d3d40a60ea06e6d669e8e5012e6c8b947a57
Successfully built langdetect


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
# !cp "/content/english_blog_corpus_dict.pickle" "/content/drive/MyDrive/blog_post_wise_dict.pickle" 
!cp '/content/drive/MyDrive/all_processed_final_min_sentence_length_dict.pickle' '/content/all_processed_final_min_sentence_length_dict.pickle'  
# !cp "/content/drive/MyDrive/blogs.zip" "/content/blogs.zip" 
# !unzip '/content/blogs.zip'

In [8]:
from bs4 import BeautifulSoup
import os 
import pandas as pd
import contractions
import nltk
from nltk.tokenize import sent_tokenize
# from spacy.lang.en.stop_words import STOP_WORDS
import pickle
from langdetect import detect
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# Extract content from post tag in blog

In [3]:
def get_file_content(path,file_names,en):
  """Returns blog content in <p> tags in html file """
  file_text = {}
  lang_exception = []
  for file_name in file_names:
    if file_name.strip().split(".")[0] in en:
      posts_list = []
      try:
        with open(path+file_name, "r",encoding='utf-8-sig',errors='ignore') as file_obj:
          contents = file_obj.read()
          soup = BeautifulSoup(contents, 'lxml')
          for tag in soup.find_all("post"):
            temp = tag.text.replace("\\", "").replace("\t",'').replace("\n",'').replace('urlLink','').lower()
            # print(temp)
            if detect(temp)== 'en':
              if len(temp.split()) > 10:
                temp = ' '.join([(contractions.fix(word)) for word in temp.split()])
                posts_list.append(temp)
      except IOError:
        print("Error: Input XML files not found ",file_name)
        pass
      except:
        print('Language detection error',file_name.strip().split(".")[0])
        lang_exception.append(file_name.strip().split(".")[0])
        pass
      # print(file_name.strip().split(".")[0])
      file_text[file_name.strip().split(".")[0]] = posts_list
  return file_text,lang_exception

In [ ]:
path = '/content/blogs/'
files_list = os.listdir(path)
total_content,lang_exception = get_file_content(path,files_list,file_chunk_content.keys())

# pickle.dump(file_chunk_content,open('en_chunk_wise_dict.pickle','wb'),protocol= pickle.HIGHEST_PROTOCOL)
len(lang_exception),len(total_content)

In [47]:
# Method to detect the non english blogs in the corpus
def remove_non_english_blog(file_dict):
  non_en = []
  for key in file_dict.keys():
    try:
      lang = detect(file_dict[key])
      if lang!= 'en':
        # print(lang,key)
        non_en.append(key)
    except :
      print('Error in Key value',key)
      non_en.extend(key)
      continue
  
  for key in non_en:
    try:
      file_dict.pop(key)
    except:
       print('Error in Key value',key)
       continue

  return file_dict

# file_chunk_content = remove_non_english_blog(file_chunk_content)

# Perform preprocessing on the extracted chunks(posts) from blogs

In [34]:

# Remove the punctuations from the sentences
def remove_punctuation(content):
  return ' '.join([token for token in content.split() if token.isalpha() == True])

# Method to perform preprocessing task
def preprocess_text(blog_text):
  sentences = nltk.sent_tokenize(blog_text)
  sentences = [remove_punctuation(sentence) for sentence in sentences]
  sentences = ' '.join(sentences)
  return sentences

In [38]:
# Perform preprocessing on the blog content
def get_preprocessed_content(file_chunk_content)
testing_blogs = {}
for index, key in enumerate(list(file_chunk_content.keys())):
  testing_blogs[key] = list(map(preprocess_text,file_chunk_content[key]))
  if index%1000 == 0:
        print('Preprocessing done for ', index)

Preprocessing done for  0
Preprocessing done for  1000
Preprocessing done for  2000
Preprocessing done for  3000
Preprocessing done for  4000
Preprocessing done for  5000
Preprocessing done for  6000
Preprocessing done for  7000
Preprocessing done for  8000
Preprocessing done for  9000
Preprocessing done for  10000
Preprocessing done for  11000
Preprocessing done for  12000
Preprocessing done for  13000
Preprocessing done for  14000
Preprocessing done for  15000
Preprocessing done for  16000
Preprocessing done for  17000
Preprocessing done for  18000
Preprocessing done for  19000


In [2]:
# preprocess_pipeline = set_preprocess_pipeline()
file_chunk_content = pickle.load(open('en_chunk_wise_dict.pickle','rb'))

In [16]:
pickle.dump(testing_blogs,open('all_processed_final_min_sentence_length_dict.pickle','wb'),protocol= pickle.HIGHEST_PROTOCOL)
!cp "/content/all_processed_final_min_sentence_length_dict.pickle" "/content/drive/MyDrive/all_processed_final_min_sentence_length_dict.pickle" 

In [9]:
testing_blogs = pickle.load(open('/content/all_processed_final_min_sentence_length_dict.pickle','rb'))

# Generate embeddings for each post (chunk) in the blog

In [10]:
import tensorflow as tf
import numpy as np
import tensorflow_hub as hub
module_url = "https://tfhub.dev/google/universal-sentence-encoder/3"
# Import the Universal Sentence Encoder's TF Hub module
embed = hub.load(module_url)

In [11]:
blog_embedding = {}
for index, key in enumerate(list(testing_blogs.keys())):
  # print(key)
  blog_embedding [key] = [embed([post])['outputs'].numpy().reshape(1,-1) for post in testing_blogs[key]]
  if index%1000 == 0:
    print('Embeddings generated for ', index)

Embeddings generated for  0
Embeddings generated for  1000
Embeddings generated for  2000
Embeddings generated for  3000
Embeddings generated for  4000
Embeddings generated for  5000
Embeddings generated for  6000
Embeddings generated for  7000
Embeddings generated for  8000
Embeddings generated for  9000
Embeddings generated for  10000
Embeddings generated for  11000
Embeddings generated for  12000
Embeddings generated for  13000
Embeddings generated for  14000
Embeddings generated for  15000
Embeddings generated for  16000
Embeddings generated for  17000


In [12]:
pickle.dump(blog_embedding,open('final_embeddings_dict.pickle','wb'),protocol= pickle.HIGHEST_PROTOCOL)
!cp "/content/final_embeddings_dict.pickle" "/content/drive/MyDrive/final_embeddings_dict.pickle" 
# blog_embedding = pickle.load(open('embeddings_dict.pickle','rb'))
# non_eng_list = pickle.load(open('non_english_list.pickle','rb'))

In [9]:
for item in non_eng_list:
  blog_embedding.pop(item)

In [13]:
query = 'automobile'
embed_query = embed([query])['outputs'][0].numpy().reshape(1,-1)

In [14]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
similarity = {}
for key in blog_embedding.keys():
  post_sim  = []
  for item in blog_embedding[key]:
    # print(item.shape,embed_query.shape)
    # print(type(item),type(embed_query))
    # print(item)
    # break
    post_sim.append(cosine_similarity(item.reshape(1,-1),embed_query))
  try:  
    similarity[key] = (np.max(post_sim),np.argmax(post_sim))
  except ValueError:
    pass
  # similarity[key] = (np.mean(post_sim))
  # break
# embed_query['outputs'][0]


In [15]:
similarity = sorted(similarity.items(), key=lambda x: x[1], reverse=True)

In [16]:
similarity[:10]

[('3724149', (0.41582772, 7)),
 ('1601416', (0.40137133, 206)),
 ('3521040', (0.36812702, 1)),
 ('3480399', (0.3669263, 50)),
 ('860293', (0.34700236, 94)),
 ('3813382', (0.34126824, 0)),
 ('3236014', (0.33970445, 94)),
 ('3647917', (0.33288568, 10)),
 ('3668410', (0.33172116, 3)),
 ('1094221', (0.33100533, 2))]

In [26]:
testing_blogs['1094221'][2]

'i never really experienced car driving during the winter time because people have told that the road are too dangerous and too i think this is perhaps the reason why my mother does not allow me to drive as much as during the i have been driving a lot i do not see any differences with my driving of driving more carfully and slowly when driving on what about the road cars are five times dirtier to compare with than the rest of the i was driving with my dirty i found out how dangerous it was to switch lanes because it was like impossible to glance at my dead spots without giving my full concentration to see right through the dirt on my i think i should really consider getting my car washed tomorrow before heading on the what is it about washing cars during winter when the minute you drive on the the car will be is not it driving with a clean car does not only means greater it also means to have better windows to see the is like a just hope the car washing place will be tomorrow'

In [ ]:
sample_sentences = ['I am always interested in reading',
                    'It is so cold outside',
                    'I purchased books','NLP is interesting','sentence encoder gives embeddings']
# query_term = []
# sample_query_embedding = embed(query_term)['outputs'][0].numpy().reshape(1,-1)
sample_sentence_embedding = [ embed([item])['outputs'][0].numpy().reshape(1,-1) for item in sample_sentences ]


In [ ]:
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

# Use PCA to reduce the dimensions of the embedding
def pca_transform(sentence_embeddings,dim):
    vectors = np.vstack((sentence_embeddings[0], sentence_embeddings[1], sentence_embeddings[2]))
    # Set the number of dimensions PCA will reduce to
    pca = PCA(n_components=dim)
    pca.fit(vectors)
    new_pca = pca.transform(vectors)
    print("original shape:   ", vectors.shape)
    print("transformed shape:", new_pca.shape)
    return(new_pca)

new_dim = pca_transform(sample_sentence_embedding, 2)

original shape:    (3, 512)
transformed shape: (3, 2)


In [ ]:
# Draw a 2d graph with the new dimensions
def two_d_graph(sample_sentences,reduced_dims):
    colors = ("red", "green", "blue")
    groups = (sample_sentences[0], sample_sentences[1], sample_sentences[2]) 
 
    # Create plot
    fig = plt.figure()
    ax = fig.gca()
 
    for data, color, group in zip(reduced_dims, colors, groups):
        x, y = data
        ax.scatter(x, y, c=color, edgecolors='none', s=30, label=group)
 
    plt.title('Plot of sentence embeddings')
    plt.legend(loc=1)
    plt.rcParams["figure.figsize"] = (8,15)
    plt.show()

two_d_graph(sample_sentences,new_dim)

In [43]:
import seaborn as sns
def plot_similarity(labels, features, rotation):
  corr = np.inner(features, features).reshape(len(labels),len(labels))
  sns.set(font_scale=0.9)
  g = sns.heatmap(
      corr,
      xticklabels=labels,
      yticklabels=labels,
      vmin=0,
      vmax=1,
      cmap="YlOrRd")
  g.set_xticklabels(labels, rotation=rotation)
  g.set_title("Semantic Textual Similarity")
  # print(corr)

def run_and_plot(sentences,embeddings):
  # message_embeddings_ = embed(messages_)
  plot_similarity(sentences, embeddings, 90)

run_and_plot(sample_sentences,sample_sentence_embedding)

NameError: ignored